Genetic algorithm to find the minimum of a three-variable function


In [3]:
#https://codereview.stackexchange.com/questions/188702/genetic-algorithm-to-find-the-minimum-of-a-three-variable-function

import numpy as np

import multiprocessing
from collections import OrderedDict
import os
import time


def eval_iter(arg_lst, l_lst):
    for c_i, args in enumerate(arg_lst):
        yield c_i, args, l_lst


def eval_func(c_i, args, l_lst):
    assert len(args) == 3
    x = args[0]
    y = args[1]
    z = args[2]
    res = x**2 + y**2 + z**2
    print(f"Eval {x}, {y}, {z}: {res}")
    l_lst[c_i] = res


if __name__ == '__main__':

    generation_num = 10
    child_num = 5

    space = OrderedDict((
        ('x', (-2., 0.)),
        ('y', (0., 2.)),
        ('z', (10., 11.))
    ))

    params = OrderedDict([(nm, []) for nm in space.keys()])
    for nm, v_range in space.items():
        params[nm] = np.random.uniform(v_range[0], v_range[1], size=child_num)

    arg_list = []
    for c_n in range(child_num):
        arg_list.append([val[c_n] for val in params.values()])

    manager = multiprocessing.Manager()
    loss_lst = manager.list([np.inf for i in range(child_num)])

    for r_n in range(generation_num):
        with multiprocessing.Pool(os.cpu_count()) as pool:
            pool.starmap(eval_func, eval_iter(arg_list, loss_lst))

        fittest_idx = int(np.argmin(loss_lst))
        base_args = arg_list[fittest_idx]
        print(f"Best {base_args}\n")

        # mutate offspring from fittest individual
        params = OrderedDict([(nm, []) for nm in space.keys()])
        for s_i, (nm, v_range) in enumerate(space.items()):
            std = (v_range[1] - v_range[0]) / 2
            noise = np.random.normal(0, std, size=child_num)
            new_param = base_args[s_i] + noise
            params[nm] = np.clip(new_param, v_range[0], v_range[1])

        arg_list = []
        for c_n in range(child_num):
            arg_list.append([val[c_n] for val in params.values()])

        loss_lst = manager.list([np.inf for i in range(child_num)])

Eval -1.6908450877830483, 1.832224533518236, 10.505708103111743: 116.58590659989393
Eval -0.0033219378320710202, 0.54018983406697, 10.032915176988846: 100.95120304075338
Eval -1.2948427389086392, 0.23209303639108048, 10.354237988363744: 108.94072921572052
Eval -0.3422251910053742, 0.058651051297697876, 10.78814008933015: 116.50452461418934
Eval -0.4431073510598631, 0.8747399981913402, 10.213662140960993: 105.28040851869898
Best [-0.0033219378320710202, 0.54018983406697, 10.032915176988846]

Eval -0.8418898104481993, 0.3369563548211088, 10.0: 100.82231803799084
Eval -0.7745680276640381, 0.06662329594375299, 10.388982386521947: 108.53534932050502
Eval 0.0, 0.03712038833300535, 10.0: 100.00137792323
Eval -0.15341630401137962, 0.6104144769512374, 10.61421591380805: 113.0577218609442
Eval -0.20409550450674654, 0.14629917534016218, 10.464180958063297: 109.56214154675956
Best [0.0, 0.03712038833300535, 10.0]

Eval 0.0, 0.0, 11.0: 121.0
Eval -0.8425625978123338, 2.0, 10.0: 104.70991173123227
E